In [1]:
import time
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random

In [2]:
ol = Overlay('/home/xilinx/jupyter_notebooks/sadd/sadd.bit') # check this path
ol.download() # this downloads your bitstream into FPGA
dma1 = ol.sadd_dma1 # first DMA. Note that we had to access the hierarchy before accessing the DMA
dma2 = ol.sadd_dma2 # second DMA
sadd_ip = MMIO(0x40000000, 0x10000) # we got this address from
xlnk = Xlnk()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: pynq.Xlnk is deprecated and will be removed in 2.7 - use pynq.allocate instead
  


In [3]:
length = 20

in_buffer1 = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer 1
in_buffer2 = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer 2
out_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # output buffer

samples = random.sample(range(0, length), length)
np.copyto(in_buffer1, samples)
samples = random.sample(range(0, length), length)
np.copyto(in_buffer2, samples)


sadd_ip.write(0x10, length) # we got this address from Vivado source. Since we didn't do port=return, and we set a constant for ap_start, we only have to write length.
sadd_ip.write(0x00, 1)
t_start = time.time()
dma1.sendchannel.transfer(in_buffer1)
dma2.sendchannel.transfer(in_buffer2)
dma1.recvchannel.transfer(out_buffer)
dma1.sendchannel.wait()
dma2.sendchannel.wait()
dma1.recvchannel.wait()
t_stop = time.time()
in_buffer1.close()
in_buffer2.close()
out_buffer.close()
print('Hardware execution time: ', t_stop-t_start)
for i in range(0, length):
    print('{}+{} = {}'.format(in_buffer1[i], in_buffer2[i], out_buffer[i]))

Hardware execution time:  0.0031807422637939453
7+13 = 20
3+0 = 3
17+7 = 24
4+8 = 12
16+16 = 32
1+1 = 2
12+14 = 26
10+4 = 14
0+2 = 2
19+12 = 31
18+5 = 23
11+11 = 22
14+3 = 17
9+9 = 18
5+17 = 22
13+15 = 28
15+10 = 25
6+19 = 25
8+6 = 14
2+18 = 20
